In [1]:
{-# LANGUAGE FlexibleContexts, GADTs, TypeOperators #-}
import Data.Manifold.TreeCover
import Data.Manifold.Web
import qualified Data.Graph as Graph
import Data.Manifold.Types
import Data.VectorSpace
import Linear.V2
import Data.AffineSpace
import Math.LinearMap.Category
import Data.Random
import Data.Random.Manifold

import Control.Arrow ((***))
import Control.Applicative (empty)
import Data.Foldable (toList, forM_)
import Data.Traversable (forM)

From [dynamic-plot](http://hackage.haskell.org/packages/dynamic-plot):

In [2]:
import Graphics.Dynamic.Plot.R2

From [diagrams](http://projects.haskell.org/diagrams):

In [3]:
import Diagrams.Prelude ( Point(P), r2, circle, (^&), (@@), _1, moveTo
                        , fc, lc, opacity, red, blue, white
                        , fromVertices, arrowBetween', arrowShaft, arc, xDir, turn
                        , arrowHead, spike, headLength, tiny, small, large, headGap
                        , with, (&), (#), (^.), (.~) )

Functions for plotting a 2D tree/web structures. Trees with lines for twigs and circles for leaves.

In [4]:
prettyTreePlot :: ShadeTree ℝ² -> [DynamicPlottable]
prettyTreePlot tr = [ plot [ shapePlot $ circle 0.03 & moveTo p & opacity 0.2 | p <- ps ]
                    , plot $ onlyNodes tr ]
 where ps = map P $ onlyLeaves_ tr
 
prettyWebPlot :: PointsWeb ℝ² () -> [DynamicPlottable]
prettyWebPlot w = [ shapePlot $ arrowBetween'
                       ( with & arrowHead .~ spike
                              & arrowShaft .~ arc xDir (1/9 @@ turn)
                              & headLength .~ tiny
                              & headGap .~ small
                       ) (P r₁) (P r₂)
                  | ((r₁@(V2 x₁ y₁),()),(r₂@(V2 x₂ y₂),())) <- edg ]
 where edg = map (gnodes *** gnodes) $ Graph.edges graph
       (graph, gnodes) = toGraph w

plotTreeAndWeb :: ShadeTree ℝ² -> IO ()
plotTreeAndWeb tr = do
    plotWindow $ prettyTreePlot tr # map (tint white)
              ++ prettyWebPlot web # map (tweakPrerendered $ opacity 0.7)
              ++ [plot [ plot p & tint red & tweakPrerendered (opacity 0.3)
                       | (p,_) <- webBoundary web ]]
    return ()
 where web = fromShadeTree (const euclideanNorm) tr

### Simple cartesian grid within a disk:

In [5]:
cartTree :: ShadeTree ℝ²
cartTree = fromLeafPoints
              [ (x^&y) | x<-[0, 0.35 .. 4]
                       , y<-[0, 0.25 .. 4]
                       , (x-2)^2 + (y-2)^2 < 4 ]

plotTreeAndWeb cartTree

![visualisation of tree-cover and points-web of cartesian-disk cloud](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/TreesAndWebs/2D-cartesiandisk.png)

### Regular (slightly stretched) hexagonal grid

In [6]:
hexagonTree = fromLeafPoints $
           [V2 x y | x<-[-2,-1.7..2], y<-[-1,-0.6..2]]
        ++ [V2 x y | x<-[-1.85,-1.55..2], y<-[-0.8,-0.4..2]]

plotTreeAndWeb hexagonTree

![visualisation of tree-cover and points-web of hexagonal points arrangement](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/TreesAndWebs/2D-hexagonal.png)

### <a id='pseudorandomCloudTree'></a>Cloud of 405 sort-of random points:

In [7]:
scatterTree = let
    tps₀ = [(0,0), (0,1), (1,1), (1,2), (2,2)]
    tps₁ = [p .+^ v^/3 | p<-tps₀, v <- [(0,0), (-1,1), (1,2)]]
    tps₂ = [p .+^ v^/4 | p<-tps₁, v <- [(0,0), (-1,1), (1,2)]]
    tps₃ = [p .+^ v^/5 | p<-tps₂, v <- [(0,0), (-2,1), (1,2)]]
    tps₄ = [p .+^ v^/7 | p<-tps₃, v <- [(0,1), (-1,1), (1,2)]]
     in fromLeafPoints $ r2<$>tps₄
plotTreeAndWeb scatterTree
((_,exampleTwiglet), exampleTwigEnviron) = twigsWithEnvirons scatterTree !! 5
plotWindow $ prettyTreePlot scatterTree
                 # map (tweakPrerendered (opacity 0.3) . tint white)
          ++ prettyTreePlot exampleTwiglet
                 # map (tint blue)
          ++ concat (prettyTreePlot . snd <$> exampleTwigEnviron)
                 # map (tweakPrerendered (opacity 0.3) . tint red)

GraphWindowSpecR2{lBound=-1.8415873015873017, rBound=3.6415873015873017, bBound=-0.5153968253968253, tBound=4.51063492063492, xResolution=640, yResolution=480}

![visualisation of tree-cover and points-wed of pseudorandom-point cloud](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/TreesAndWebs/2D-scatter.png)
![visualisation of a lowest-level twigs and its neighbours in the tree-cover of a pseudorandom-point cloud](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/TreesAndWebs/2D-scatter_twig-environs.png)

### 100 actually-random, normally-distributed points<!--, with focus on one local point-group and its neighbours-->:

In [8]:
randomTr <- fmap fromLeafPoints . forM [0..100] $ \_->
     runRVar (sample $ (1^&1):±[1^&0, 0^&1]) StdRandom :: IO ℝ²

-- ((_,twig),envi) <- runRVar (randomElement $ twigsWithEnvirons randomTr) StdRandom
-- plotWindow $ prettyTreePlot randomTr
--          ++ prettyTreePlot twig
--          ++ [plot $ prettyTreePlot.snd<$>envi]

plotTreeAndWeb randomTr

![visualisation of tree-cover and points-web of pseudorandom-point cloud](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/TreesAndWebs/2D-normaldistrib.png)

In [9]:
cartWeb :: PointsWeb ℝ² ()
cartWeb = fromShadeTree (const euclideanNorm) cartTree

[(x, nearestNeighbour cartWeb (x^&1.6)) | x<-[1.7, 1.8 .. 2.2]]

[(1.7,Just (V2 1.7499999999999996 1.5,())),(1.8,Just (V2 1.7499999999999996 1.5,())),(1.9000000000000001,Just (V2 1.7499999999999996 1.5,())),(2.0,Just (V2 2.0999999999999996 1.5,())),(2.0999999999999996,Just (V2 2.0999999999999996 1.5,())),(2.1999999999999993,Just (V2 2.0999999999999996 1.5,()))]